In [188]:
%%time
# -*- coding: utf-8 -*-
import codecs
def getVerticeList(data,flag):
    """This function is used to obtain the list which shows all the vertices in graph"""
    file = open(data)
    text = file.read().decode("utf-8")
    file.close()
    text = text.split('\n')
    verticeList = []
    outverticeList=[]
    for i in text:
        sentence = i.split(' ')
        verticeList.append(str(sentence[0]))
        outverticeList.append(str(sentence[0]))
        verticeList.append(str(sentence[1]))
    verticeList = list(verticeList)
    outverticeList = list(set(outverticeList))
    addvertice=[]
    for i in verticeList:
        if i not in outverticeList:
            addvertice.append(i)
    if flag == 'a':
        return verticeList
    else :
        return addvertice

def splitVE(line):
    """This function split a row of data into two parts: vertice and edges
       The output is a tuple which contains a string of vertice name and a list of it's outward edges
    """
   # line = str(cline).split(' ')
    line = str(line).split(' ')
    ver = line[0]
    edges = [(line[1], float(line[2]))]
    return (ver, edges)

def fillINF(line, data):
    """This function will fill in the blanks in an edges list
       That is fill all the target vertice with weight of 'infinity' into the list
       If the target vertice is identical to the input vertice, fill in the weight with 0
    """
    edges = line[1]
    needToFill = getVerticeList(data,'a')[:]
    print(needToFill)
    for i in edges:
        needToFill.remove(i[0])
    for j in needToFill:
        if line[0] == j:
            line[1].append((j, 0))
        else:
            line[1].append((j, float("+inf")))
    outputEdges = line[1]
    outputEdges = sorted(outputEdges, key=lambda x: x[0])
    return (line[0],outputEdges)

def fillLENGTH(vertice, start):
    """This function can be used to add the path weight from start to the current vertex
       Since it's the first step, if start is the same with vertice, the path is 0
       otherwise, fill in 'infinity'
    """
    edge =vertice[1][:]
    ver = vertice[0]
    if start == ver:
        return (ver, (0, edge))
    else:
        return (ver, (float("+inf"), edge))

def getOriginalMatrix(data, start):
    """This function returns a complete path adjacency matrix of the start node
       Each line stores three parts:
       The first is a vertex name means 'from start to this vertex'
       Then the second part store the length from start to this vertex
       The final part stores from this vertex to all other vertices with the corresponding weight
       """
    #data2="result.txt"
    graph = sc.textFile(data)
    formedGraph = graph.map(lambda x: splitVE(x))
    addvertice= getVerticeList(data,'b')
    newrdd=[]
    for i in addvertice:
        newrdd.append((i,[(i,0.0)]))
    formedGraph.collect()  
    newrdd=sc.parallelize(newrdd,4)
    newrdd.cache()
    formedGraph=formedGraph.union(newrdd)
    filledGraph = formedGraph.reduceByKey(lambda x, y: x+y) 
    filledINFGraph = filledGraph.map(lambda x: fillINF(x, data))
    filledLENGraph = filledINFGraph.map(lambda x: fillLENGTH(x, start))
    filledLENGraph.cache()
    return filledLENGraph

def getAdjDict(data, start):
    """This function calculates the dictionary within values of outward edges and weights,
       The key of this dictionary is the start
    """
    adj = getOriginalMatrix(data, start).collect()
    adjDict = dict()
    for i in adj:
        adjDict[i[0]] = i[1][1]
    return adjDict

def newPath(vertice, adjDict):
    """This function update the weights to the length from start to B via A
       Where A and B can be all the vertices in graph
    """
    known = vertice[1][0]
    edges = vertice[1][1][:]
    new = []
    for i in edges:
        newPath = i[1]+known
        new.append((i[0], (newPath, adjDict[i[0]])))
    return new

CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 23.1 µs


In [189]:
def shortestPath(data, start):
    """This part is the core of Bell-Ford Shortest Path Algorithm, 
       from the start to the end, it compare all the known weights of path via other nodes
       choosing the shortest one and update, and follow the loops again.
       The loop will stop when there are no path be updated
    """
    verticeList = getVerticeList(data,'a')
    originalMatrix = getOriginalMatrix(data, start)
    adjDictionary = getAdjDict(data, start)
    flag = 0
    
    update = True
    origin = getOriginalMatrix(data, start)
    new = getOriginalMatrix(data, start)
    adj = getAdjDict(data, start)
    while update:
        flag += 1
        origin = new
        newMAPGraph = new.flatMap(lambda x: newPath(x, adj))
        #print(newMAPGraph.collect())
        new = newMAPGraph.reduceByKey(lambda x, y: x if x[0]<=y[0] else y) 
        #print('ssssssssss')
        #print(new.collect()) 
        origin.cache()
        new.cache()
        if new.collect() == origin.collect():
            update = False
        elif flag == len(adjDictionary)-1:
            update = False
    shortestList = new.map(lambda x: (x[0], x[1][0])).collect()
    return sorted(shortestList, key = lambda x: x[0])



In [190]:
shortestPath('smalldemo.txt', '1')

[('AbGhurayb,Iraq', 622.5),
 ('Abadan,Iran', 0.0),
 ('Abha,SaudiArabia', 2393.1),
 ('AbuDhabi,UnitedArabEmirates', 1433.09),
 ('AdDnyah,Iraq', 444.8),
 ('Adana,Turkey', 1807.41),
 ('Aden,Yemen', 3266.91),
 ('Adyaman,Turkey', 1584.88)]

In [103]:
shortestPath('testGraph.txt', 'B')

[('1', inf), ('2', inf), ('3', inf)]

In [10]:
shortestPath('data/testGraph.txt', '5')

IOError: [Errno 2] No such file or directory: 'data/testGraph.txt'

In [ ]:
fileObject = open('world.txt', 'w')
for ip in final:
        "    fileObject.write(ip)\n",
    "    fileObject.write('\\n')\n",
    "fileObject.close()"

In [120]:
file = open("input.txt")
text = file.read().decode("utf-8")
file.close()
text = text.split('\n')
a=text[3].split(' ')
str(a[1])

UnicodeEncodeError: 'ascii' codec can't encode character u'\u016b' in position 2: ordinal not in range(128)

In [209]:
# url의 파일을 유니코드 인코딩으로 읽음
textFile = sc.textFile("input.txt", use_unicode=True)
# utf-8 인코딩을 이용하여 처리
counts = textFile.flatMap(lambda line: str(line.encode('utf-8')).split("\n"))\
          .map(lambda line: (line.split("\t")[0], 1))\
          .reduceByKey(lambda a, b: a + b)
# hdfs에 result 폴더에 저장
#counts.saveAsTextFile("result.txt")
counts.collect()


[('Abadan,Iran Bah\xc3\xa7elievler,Turkey 1678.7', 1),
 ('Abadan,Iran Ansan-si,SouthKorea 9477.22', 1),
 ('Abadan,Iran AdD\xc4\xabw\xc4\x81n\xc4\xabyah,Iraq 444.8', 1),
 ('Abadan,Iran BandarLampung,Indonesia 0.0', 1),
 ('Abadan,Iran Abha,SaudiArabia 2393.1', 1),
 ('Abadan,Iran Ajmer,India 4139.16', 1),
 ('Abadan,Iran Akashi,Japan 10724.94', 1),
 ('Abadan,Iran Ankara,Turkey 1665.9', 1),
 ('Abadan,Iran Akola,India 4986.06', 1),
 ('Abadan,Iran AlA\xe1\xb8\xa9mad\xc4\xab,Kuwait 269.25', 1),
 ('Abadan,Iran AlKharj,SaudiArabia 1141.93', 1),
 ('Abadan,Iran Al\xe1\xb8\xa8illah,Iraq 532.44', 1),
 ('Abadan,Iran Baku,Azerbaijan 1996.23', 1),
 ('Abadan,Iran Andijon,Uzbekistan 3300.46', 1),
 ('Abadan,Iran Bangkok,Thailand 10870.33', 1),
 ('Abadan,Iran Almaty,Kazakhstan 3857.16', 1),
 ('Abadan,Iran Ahmadnagar,India 5113.66', 1),
 ('Abadan,Iran Aden,Yemen 3266.91', 1),
 ('Abadan,Iran Ardab\xc4\xabl,Iran 1635.73', 1),
 ('Abadan,Iran AlorSetar,Malaysia 11940.61', 1),
 ('Abadan,Iran Ageoshimo,Japan 1134

NameError: name 'graph' is not defined

In [204]:
def splitVE(line):
    """This function split a row of data into two parts: vertice and edges
       The output is a tuple which contains a string of vertice name and a list of it's outward edges
    """
   # line = str(cline).split(' ')
    line = line.split(' ')
    ver = line[0]
    edges = [(line[1], float(line[2][:-2]))]
    return (ver, edges)
    #return line[2][:-2]
data2="result.txt"
graph = sc.textFile(data2)
formedGraph = graph.map(lambda x: splitVE(x))

In [205]:
formedGraph.collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/chris/Downloads/5003/result.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:204)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:251)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:251)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:253)
	at org.apache.spark.rdd.RDD$$anonfun$partitions

In [200]:
file = open("input.txt")
text = file.read().decode("utf-8")
file.close()
text = text.split('\n')
verticeList = []
outverticeList=[]
for i in text:
    sentence = i.split(' ')
    print(sentence)
    verticeList.append(str(sentence[0]))
    outverticeList.append(str(sentence[0]))
    verticeList.append(str(sentence[1]))
verticeList = list(verticeList)
outverticeList = list(set(outverticeList))

[u'Abadan,Iran', u'Abadan,Iran', u'0.0']
[u'Abadan,Iran', u'Abha,SaudiArabia', u'2393.1']
[u'Abadan,Iran', u'AbuDhabi,UnitedArabEmirates', u'1433.09']
[u'Abadan,Iran', u'AbGhurayb,Iraq', u'622.5']
[u'Abadan,Iran', u'AdDnyah,Iraq', u'444.8']
[u'Abadan,Iran', u'Adana,Turkey', u'1807.41']
[u'Abadan,Iran', u'Aden,Yemen', u'3266.91']
[u'Abadan,Iran', u'Adyaman,Turkey', u'1584.88']
